In [2]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
head = pd.read_csv('credit_names.csv', header=None)
print(head)


                                                    0
0   1. Title: Japanese Credit Screening (examples ...
1                              2. Source information:
2                          -- Creators: Chiharu Sano 
3                              -- Donor: Chiharu Sano
4                            csano@bonnie.ICS.UCI.EDU
5                                    -- Date: 3/19/92
6                                     3. Past usage: 
7                                   -- None Published
8                            4. Relevant information:
9      --  Examples represent positive and negativ...
10                                    granted credit.
11     --  The theory was generated by talking to ...
12                                            credit.
13                        5. Number of instances: 125


In [4]:
df = pd.read_csv('credit.csv')
print(df.head())

   b  30.83      0  u  g  w  v  1.25  t t.1  01  f g.1  00202  0.1  +
0  a  58.67  4.460  u  g  q  h  3.04  t   t   6  f   g  00043  560  +
1  a  24.50  0.500  u  g  q  h  1.50  t   f   0  f   g  00280  824  +
2  b  27.83  1.540  u  g  w  v  3.75  t   t   5  t   g  00100    3  +
3  b  20.17  5.625  u  g  w  v  1.71  t   f   0  f   s  00120    0  +
4  b  32.08  4.000  u  g  m  v  2.50  t   f   0  t   g  00360    0  +
